## Imports

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import folium
from folium import FeatureGroup, LayerControl, Map, Marker
import pandas as pd
import random
from scipy import stats
from collections import Counter

%matplotlib inline

## Consts

In [10]:
#cols in the original table
PROVIDER_AND_ID = 'provider_and_id'
YEAR = 'accident_year'
LAT = 'latitude'
LONG = 'longitude'
X = 'x'
Y = 'y'
SEVERITY = 'injury_severity_hebrew'
ROAD_SEGMENT_NAME = 'road_segment_name'
ROAD_SEGMENT = 'road_segment_id'
ROAD_SEGMENT_LENGTH = 'road_segment_length_km'
SEVERITY_DEAD = 'הרוג'
SEVERITY_HARD = 'פצוע קשה'
ID = 'accident_id'
PROVIDER_CODE = 'provider_code'
KM_LOCATION = 'km'
SEGMENT_FROM_KM = 'segment_from_km'
ROAD1 = 'road1'
ROAD2 = 'road2'
LOC_ACCURACY = 'location_accuracy'


RELEVANT_KEYS_ANALYSIS = [PROVIDER_AND_ID, PROVIDER_CODE, ID, YEAR, ROAD_SEGMENT_NAME, ROAD_SEGMENT, ROAD_SEGMENT_LENGTH, KM_LOCATION,
                          LAT, LONG, X, Y, SEVERITY, ROAD1, ROAD2, LOC_ACCURACY]

# new cols
KM_FROM_MEDIAN = 'km_from_median'
IS_OUTLIER = 'is_outlier'


DEFAULT_ZOOM = 9
FROM_YEAR = 2016
DEFAULT_COORD = (32.079184, 34.824768)
MEDIAN = 'MEDIAN'

## Load data

In [4]:
csv_path = r'views2021/involved_markers_hebrew.csv'
data = pd.read_csv(csv_path, na_values='')

C:\Users\dinasv\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (36,38,40,44,92,94,137,151,160) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
data.head()

,accident_id,provider_and_id,provider_code,file_type_police,involved_type,involved_type_hebrew,license_acquiring_date,age_group,age_group_hebrew,sex,...,vehicle_status_hebrew,vehicle_attribution,vehicle_attribution_hebrew,seats,total_weight,total_weight_hebrew,vehicle_vehicle_type,vehicle_vehicle_type_hebrew,vehicle_damage,vehicle_damage_hebrew
0,2008000002,32008000002,3,NaN,3,נפגע,0,4,15-19,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2008000002,32008000002,3,NaN,1,נהג,0,99,לא ידוע,NaN,...,NaN,1.0,ישראלי,1.0,2.0,2.0-2.9,1.0,רכב נוסעים פרטי,NaN,NaN
2,2008000004,12008000004,1,NaN,3,נפגע,0,99,לא ידוע,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2008000004,12008000004,1,NaN,1,נהג,1978,11,50-54,1.0,...,NaN,1.0,ישראלי,99.0,3.0,3.0-3.5,2.0,משא עד 4 טון - אחוד (טרנזיט),NaN,NaN
4,2008000006,12008000006,1,NaN,3,נפגע,0,6,25-29,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
csv_path = r'road_segments_table.csv'
seg_info = pd.read_csv(csv_path, na_values='', index_col=1)

In [8]:
seg_info.head()

,id,road,segment,from_km,from_name,to_km,to_name
segment_id,,,,,,,
10010,1,1,10,0.0,מחלף קיבוץ גלויות,4.6,מחלף גנות
10020,2,1,20,4.6,מחלף גנות,6.4,מחלף שפירים
10030,3,1,30,6.4,מחלף שפירים,13.1,מחלף בן גוריון
10040,4,1,40,13.1,מחלף בן גוריון,14.5,מחלף לוד
10050,5,1,50,14.5,מחלף לוד,18.9,מחלף בן שמן


In [11]:
def filter_data(data):
    # get only relevant data
    filtered = data[RELEVANT_KEYS_ANALYSIS].drop_duplicates(subset=[PROVIDER_AND_ID]).dropna(subset=[LAT, LONG, 
                                                                                                     ROAD_SEGMENT, KM_LOCATION])
    # from this year the data was corrected
    filtered = filtered[filtered[YEAR] >= FROM_YEAR]

    types = {ROAD_SEGMENT : 'int64', PROVIDER_AND_ID: 'str', ID:'str', PROVIDER_CODE: 'str', KM_LOCATION : 'int64', 
             ROAD1: 'int64'}

    filtered = filtered.astype(types)
    filtered[SEGMENT_FROM_KM] = filtered[ROAD_SEGMENT].map(seg_info['from_km'])
    
    return filtered

filtered = filter_data(data)
filtered.head()

,provider_and_id,provider_code,accident_id,accident_year,road_segment_name,road_segment_id,road_segment_length_km,km,latitude,longitude,x,y,injury_severity_hebrew,road1,road2,location_accuracy,segment_from_km
1286801,32016000011,3,2016000011,2016,מחלף שפירים - מחלף בן גוריון,10030,6.7,64,32.007843,34.836422,184677.0,657240.0,NaN,1,412.0,1,6.4
1286823,32016000028,3,2016000028,2016,מחלף מבוא איילון - מחלף קריית ראשון,4310006,1.0,100,31.966090,34.763045,177725.0,652636.0,NaN,431,0.0,1,9.5
1286827,12016000032,1,2016000032,2016,צומת רמה מערב - צומת חסן,8640010,14.0,120,32.995696,35.304508,228802.0,766735.0,פצוע קל,864,0.0,1,0.0
1286828,32016000033,3,2016000033,2016,צומת אבליים - צומת יבור,700100,8.9,550,32.825660,35.152530,214589.0,747874.0,פצוע קל,70,0.0,1,54.4
1286833,32016000037,3,2016000037,2016,צומת האלה - כניסה לרמת בית שמש (דרום),380020,5.0,100,31.687559,34.946399,194984.0,621695.0,פצוע קל,38,0.0,1,10.0


Save a filtered file for faster loading

In [12]:
filtered.to_csv(r'views2021/involved_markers_hebrew_filtered.csv')

In [13]:
filtered[filtered[ROAD_SEGMENT] == 10010]

,provider_and_id,provider_code,accident_id,accident_year,road_segment_name,road_segment_id,road_segment_length_km,km,latitude,longitude,x,y,injury_severity_hebrew,road1,road2,location_accuracy,segment_from_km
1289799,32016002008,3,2016002008,2016,מחלף קיבוץ גלויות - מחלף גנות,10010,4.6,10,32.038875,34.792774,180566.0,660696.0,פצוע קל,1,0.0,1,0.0
1292838,32016004114,3,2016004114,2016,מחלף קיבוץ גלויות - מחלף גנות,10010,4.6,30,32.027302,34.809022,182096.0,659407.0,פצוע קל,1,0.0,1,0.0
1293992,32016004927,3,2016004927,2016,מחלף קיבוץ גלויות - מחלף גנות,10010,4.6,30,32.027302,34.809022,182096.0,659407.0,פצוע קל,1,0.0,1,0.0
1297592,32016007319,3,2016007319,2016,מחלף קיבוץ גלויות - מחלף גנות,10010,4.6,40,32.021411,34.817029,182850.0,658751.0,NaN,1,0.0,1,0.0
1298702,32016008025,3,2016008025,2016,מחלף קיבוץ גלויות - מחלף גנות,10010,4.6,30,32.027302,34.809022,182096.0,659407.0,פצוע קל,1,0.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964721,32021087819,3,2021087819,2021,מחלף קיבוץ גלויות - מחלף גנות,10010,4.6,0,31.799843,35.114516,210934.0,634120.0,פצוע קל,1,0.0,3,0.0
1966435,32021093490,3,2021093490,2021,מחלף קיבוץ גלויות - מחלף גנות,10010,4.6,20,32.033102,34.800909,181332.0,660053.0,פצוע קל,1,0.0,1,0.0
1966917,32021095001,3,2021095001,2021,מחלף קיבוץ גלויות - מחלף גנות,10010,4.6,0,31.799843,35.114516,210934.0,634120.0,פצוע קל,1,0.0,3,0.0
1967005,12021095311,1,2021095311,2021,מחלף קיבוץ גלויות - מחלף גנות,10010,4.6,0,31.799843,35.114516,210934.0,634120.0,פצוע קל,1,0.0,3,0.0


In [14]:
len(filtered)

61870

Leave only accidents with accurate location

In [15]:
filtered_accurate_coords = filtered[filtered[LOC_ACCURACY] == 1]
len(filtered_accurate_coords)

61118

## Plot funcs

In [16]:
def create_map(coord):
    
    folium_map = folium.Map(location=coord, zoom_start=DEFAULT_ZOOM)
    
    return folium_map

In [17]:
def plot_coord(folium_map, coord, count, color, icon):
   
    tooltip = 'Click to see accident counts'
    folium.Marker(coord, icon=folium.Icon(color=color, icon=icon), 
                  popup=f'<i>{count}</i>', tooltip=tooltip).add_to(folium_map)

In [18]:
def plot_all_coords(coords, folium_map, color='green', icon='ok-sign'):
    """
    Plot all coords according to location and count
    """

    coord_to_count = coords.groupby([LAT, LONG]).size()
    
    for coord, count in coord_to_count.items():
        plot_coord(folium_map, coord, count, color=color, icon=icon)
    
    return coord_to_count

In [19]:
def plot_median_coord(median_coord, coord_to_count, folium_map, color, icon):
    
    count = coord_to_count[median_coord] if median_coord in coord_to_count else 1
    plot_coord(folium_map, median_coord, count, color=color, icon=icon)

In [20]:
def plot_all_segment_coords(segment_rows, outlier_rows, median_coord, segment_map_layer, decimals=4):
    """
    Plot all coords in segment, rounded to decimals, according to their location and count
    Median is blue, outliers are red, all other coords are green
    
    Keyword arguments:
    segment_rows -- segment rows without outliers
    outlier_rows -- outlier segment rows
    segment_map_layer -- points are added to this layer
    decimals -- round to decimals, very close points are merged 
                to the same point with a larger count
    """
    
    segment_coords = segment_rows[[LAT, LONG]].dropna().round(decimals=decimals)
    outlier_coords = outlier_rows[[LAT, LONG]].dropna().round(decimals=decimals)
    
    coord_to_count = plot_all_coords(segment_coords, segment_map_layer, color='green', icon='ok-sign')
    
    median_coord = tuple(np.around(median_coord, decimals))
    plot_median_coord(median_coord, coord_to_count, segment_map_layer, color='blue', icon='screenshot')
    
    plot_all_coords(outlier_coords, segment_map_layer, color='red', icon='exclamation-sign')


In [21]:
def get_segment_rows(data, road_segment_id, max_coords=None):
    
    segment_rows = data[data[ROAD_SEGMENT]==road_segment_id]
    
    return segment_rows[:max_coords]

In [22]:
def plot_outliers(outlier_segments, max_coords=500):
    
    outlier_map = create_map(DEFAULT_COORD)
    
    for road_segment_id, segment_rows in outlier_segments.groupby(ROAD_SEGMENT):
        
        first_row = segment_rows.iloc[0]
        segment_name = first_row[ROAD_SEGMENT_NAME]
        segment_length = round(first_row[ROAD_SEGMENT_LENGTH], 1)
        road_segment_id = int(road_segment_id)
        
        outliers = segment_rows[segment_rows[IS_OUTLIER] == True]
        not_outliers = segment_rows[segment_rows[IS_OUTLIER] == False]
        median_row = segment_rows[segment_rows[IS_OUTLIER] == MEDIAN].iloc[0]
        median_coord = (median_row[LAT], median_row[LONG])
        
        segment_map_layer = FeatureGroup(name=f'{road_segment_id} {segment_length} km {segment_name}', show=False)
        plot_all_segment_coords(not_outliers, outliers, median_coord, segment_map_layer)
        
        segment_map_layer.add_to(outlier_map)
        
    LayerControl().add_to(outlier_map)
    
    return outlier_map

## Find outliers by km from median

In [23]:
def spherical_distance(lat1, long1, lat2, long2):
    """
    Calculate the spherical distance in km between two coordinates in WGS-84 using Vincenty's formulae
    
    credit: https://www.johndcook.com/blog/2018/11/24/spheroid-distance/
    """

    lat1, long1, lat2, long2 = np.deg2rad(lat1), np.deg2rad(long1), np.deg2rad(lat2), np.deg2rad(long2)
    
    phi1 = 0.5*np.pi - lat1
    phi2 = 0.5*np.pi - lat2
    r = 0.5*(6378137 + 6356752) # mean radius in meters
    t = np.sin(phi1)*np.sin(phi2)*np.cos(long1-long2) + np.cos(phi1)*np.cos(phi2)
    
    # -1<=t<=1 for the arcos func
    t = np.minimum(t, 1)
    t = np.maximum(t, -1)
    
    return round(r * np.arccos(t) / 1000, 2)

In [24]:
def is_far_from_median(data, row, outliers, segment_length, padding):
    """
    Check if the distance of row from the median_coord is > segment_length + padding*segment_length
    The median is computed according to all coords, after removing the outliers and the current coord
    """
    
    curr_coord = [row[LAT], row[LONG]]

    data_without_outlier = data[~data[PROVIDER_AND_ID].isin(outliers) & data[PROVIDER_AND_ID] != row[PROVIDER_AND_ID]]
    
    data_for_median = data_without_outlier
    if data_without_outlier.empty: 
            data_for_median = data
            
    median_coord = np.median(data_for_median[[LAT, LONG]], axis=0)

    dist = spherical_distance(*curr_coord, *median_coord)

    return dist > segment_length + padding*segment_length

In [25]:
def get_segment_outliers_far_from_median(data, segment_length, padding):
    
    outliers = []
    
    for i, row in data.iterrows():
        if is_far_from_median(data, row, outliers, segment_length, padding):
            outliers.append(row[PROVIDER_AND_ID])
            
    return outliers

In [26]:
def km_from_median_col(segment_rows, median_coord):
    
     return segment_rows.apply(lambda row: spherical_distance(row[LAT], row[LONG], *median_coord), 
                                                                axis=1)

In [27]:
def median_row(seg, segment_length, segment_name, median_coord):
    return {ROAD_SEGMENT: seg, ROAD_SEGMENT_LENGTH: segment_length, ROAD_SEGMENT_NAME: segment_name, 
            LAT: median_coord[0], LONG: median_coord[1], IS_OUTLIER: MEDIAN, KM_FROM_MEDIAN: 0}

In [28]:
def updated_segment_rows(seg, segment_length, segment_name, segment_rows, outliers):
    
    segment_rows = segment_rows.copy(deep=True)
    
    segment_rows[IS_OUTLIER] = np.where(segment_rows[PROVIDER_AND_ID].isin(outliers), True, False)

    not_outlier_rows = segment_rows[~segment_rows[IS_OUTLIER]]
    
    data_for_median = not_outlier_rows
    if not_outlier_rows.empty: 
        data_for_median = segment_rows
            
    median_coord = np.median(data_for_median[[LAT, LONG]], axis=0)
    
    segment_rows[KM_FROM_MEDIAN] = km_from_median_col(segment_rows, median_coord)
    
    segment_rows = segment_rows.append(median_row(seg, segment_length, segment_name, median_coord), ignore_index=True);
    
    return segment_rows

In [29]:
def get_outliers_by_km(data, min_sample_size=2, padding=0.25):
    
    res = []
    
    for seg, segment_rows in data.groupby(ROAD_SEGMENT):
        
        if len(segment_rows) <= min_sample_size:
            continue
            
        segment_length = segment_rows.iloc[0][ROAD_SEGMENT_LENGTH]
        segment_name = segment_rows.iloc[0][ROAD_SEGMENT_NAME]
        outliers = get_segment_outliers_far_from_median(segment_rows, segment_length, padding)
        
        if len(outliers) > 0:
            
            segment_rows = updated_segment_rows(seg, segment_length, segment_name, segment_rows, outliers)
            
            res.append(segment_rows)
            
            
    return pd.concat(res) if res else pd.DataFrame()

## Pick minimum sample size

In [31]:
MIN_SAMPLE_SIZE = 10

In [48]:
MIN_SAMPLE_SIZE = 3

## Unreliable segments
**Most of the coords are wrong, therefore it is not possible to find outliers - the median is in the wrong coord**

440030 צומת גזר - מחלף רמלוד

**Wrong segment length**

Should be 4.5 km, segment_length=0.8   
550030 צומת כפר סבא (מזרח) - צומת לאלפי מנשה

Should be 1.9 km, segment_length=1.2   
צומת חדרה (מזרח) - צומת אלון (שמורת אלון) 650030  	

## Calculate all outliers for accurate coords

In [49]:
outliers = get_outliers_by_km(filtered_accurate_coords, MIN_SAMPLE_SIZE)
len(outliers)

381

In [50]:
segs_with_outliers = outliers[ROAD_SEGMENT].unique()
len(segs_with_outliers)

6

In [51]:
outliers_no_median = outliers[outliers[IS_OUTLIER]!=MEDIAN]

In [52]:
assert all((outliers_no_median[KM_LOCATION]/10)<=(outliers_no_median[SEGMENT_FROM_KM]+outliers_no_median[ROAD_SEGMENT_LENGTH]))

In [53]:
assert all(outliers_no_median[SEGMENT_FROM_KM] <= (outliers_no_median[KM_LOCATION]/10))

In [54]:
outliers[(outliers[ROAD_SEGMENT] == segs_with_outliers[1])&(outliers[IS_OUTLIER] == True)]

,provider_and_id,provider_code,accident_id,accident_year,road_segment_name,road_segment_id,road_segment_length_km,km,latitude,longitude,x,y,injury_severity_hebrew,road1,road2,location_accuracy,segment_from_km,is_outlier,km_from_median
16,32020079905,3,2020079905,2020.0,צומת כפר סבא (מזרח) - צומת לאלפי מנשה,550030,0.8,77.0,32.172662,34.958495,196253.0,675484.0,NaN,55.0,0.0,1.0,7.0,True,1.41


In [66]:
outlier_map = plot_outliers(outliers)
outlier_map

Note that the algorithm can't detect outliers that are very close to the start or the end of the segment.    

## Save output

In [58]:
len(filtered_accurate_coords), len(outliers[outliers[IS_OUTLIER]==True])

(61118, 37)

In [57]:
unreliable_segment_ids = [550030]

In [63]:
outliers_output = outliers.copy()
outliers_output = outliers_output[outliers_output[ROAD_SEGMENT] != 550030]

# ['unreliable_segment'] = np.where(outliers[ROAD_SEGMENT].isin(unreliable_segment_ids), True, False)
# outliers.groupby('unreliable_segment').size()

381


In [64]:
outliers_output.to_csv("outliers_2021.csv", header=True, index=False, encoding='utf-8')

In [65]:
path = r'segments//'

on_segment_col = 'on_segment'
outlier = 'מחוץ למקטע'
not_outlier = 'על המקטע'
median = 'חציון'

for road_segment_id, segment_rows in outliers_output.groupby(ROAD_SEGMENT):
    
    segment_rows = segment_rows.copy()
    segment_rows.loc[segment_rows[IS_OUTLIER] == True, on_segment_col] = outlier
    segment_rows.loc[segment_rows[IS_OUTLIER] == False, on_segment_col] = not_outlier
    segment_rows.loc[segment_rows[IS_OUTLIER] == MEDIAN, on_segment_col] = median
    
    segment_rows.to_csv(f'{path}{road_segment_id}.csv', header=True, index=False, encoding='utf-8')
    

In [67]:
outlier_map.save('outliers_2021.html')